# 测试训练好的模型的决策效果

In [2]:
import matplotlib.pyplot as plt
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import PPO,DDPG,DQN,A2C
from stable_baselines3.common.vec_env.dummy_vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
import gym
import numpy as np
import time
from stable_baselines3.common.callbacks import EvalCallback
import env
env = env.ElectricVehicleEnv()
#vec_env = make_vec_env(env, n_envs=4)
#check_env(env)

In [2]:
import numpy as np
from env import ElectricVehicleEnv
from stable_baselines3 import A2C
# 加载训练好的模型
#model = A2C.load("best_model/A2C/best_model.zip")
model = DQN.load("best_model/DQN/best_model.zip")
#model = A2C.load("model/a2c_electric_vehicle.zip")
#model = DQN.load("model/dqn_electric_vehicle.zip")
def print_state(state):
    """
    打印状态的每个属性，并标注其含义。
    """
    print(f"  当前节点 (Current Node): {state[0]}")
    print(f"  当前电池电量 (Battery): {state[1]*2}")
    print(f"  剩余时间 (Remaining Time): {state[2]/25}")
    print(f"  目标节点 (Target Node): {state[3]}")

def print_step_details(info):
    """
    打印每个步骤的详细信息。
    """
    print(f"  从节点 (From Node): {info['from_node']} 到节点 (To Node): {info['to_node']}")
    print(f"  距离缩短 (Distance_decrease): {info['distance_decrease']} km")
    print(f"  耗电 (Energy Consumed): {info['energy_consumed']} kWh")
    print(f"  是否充电 (Charging): {'Yes' if info['charging'] else 'No'}")
    if info['charging']:
        print(f"  充电量 (Energy Charged): {info['energy_charged']} kWh")
    print(f"  电量从 (Battery Start): {info['battery_start']} kWh 变成 (Battery End): {info['battery_end']} kWh")
    print(f"  剩余时间从 (Time Start): {info['time_start']} h 变成 (Time End): {info['time_end']} h")

# 测试模型

In [5]:

# 测试训练好的模型
obs, info = env.reset()
done = False

print("Initial State:")
print_state(obs)
step = 0
# 执行100步动作
for i in range(100):
    # 预测动作
    action, _states = model.predict(obs, deterministic=True)
    # 执行动作
    obs, reward, done,done1,  info = env.step(action)

    print("-" * 30)
    print(f"\nStep {step + 1}:")
    step+=1
    print_step_details(info)
    print_state(obs)
    print("Reward:", reward)
    print("Done:", done)
    
    if done:
        print("Episode finished! Resetting environment.")
        for i in range(50):
            obs, info = env.reset(seed=i)
        print("Initial State:"+"-"*10)
        print_state(obs)
        step = 0


Initial State:
  当前节点 (Current Node): 26.0
  当前电池电量 (Battery): 34.0
  剩余时间 (Remaining Time): 1.7917361450195313
  目标节点 (Target Node): 40.0
------------------------------

Step 1:
  从节点 (From Node): 26 到节点 (To Node): 40
  距离缩短 (Distance_decrease): 16.74243852 km
  耗电 (Energy Consumed): 2.0090926224 kWh
  是否充电 (Charging): No
  电量从 (Battery Start): 34.0 kWh 变成 (Battery End): 31.9909073776 kWh
  剩余时间从 (Time Start): 1.791736114 h 变成 (Time End): 1.4927639975714286 h
  当前节点 (Current Node): 40.0
  当前电池电量 (Battery): 31.990907669067383
  剩余时间 (Remaining Time): 1.4927639770507812
  目标节点 (Target Node): 40.0
Reward: -68.0090926224
Done: True
Episode finished! Resetting environment.
Initial State:----------
  当前节点 (Current Node): 24.0
  当前电池电量 (Battery): 33.0
  剩余时间 (Remaining Time): 1.5200991821289063
  目标节点 (Target Node): 25.0
------------------------------

Step 1:
  从节点 (From Node): 24 到节点 (To Node): 25
  距离缩短 (Distance_decrease): 14.02364708 km
  耗电 (Energy Consumed): 1.4023647080000001 kWh
  是

In [6]:
import logging
import os
import numpy as np
from env import ElectricVehicleEnv
from stable_baselines3 import A2C, DQN

# 设置绝对路径，确保文件生成在当前目录中
log_file_path = os.path.join(os.getcwd(), 'model_output.log')

# 配置日志，确保写入文件
logging.basicConfig(filename=log_file_path, level=logging.INFO, format='%(message)s', filemode='w')

def log_info(*args):
    """
    替代print函数，用logging.info来保存日志。
    """
    message = " ".join(map(str, args))
    print(message)
    logging.info(message)

def print_state(state):
    """
    打印状态的每个属性，并标注其含义。
    """
    log_info(f"  当前节点 (Current Node): {state[0]}")
    log_info(f"  当前电池电量 (Battery): {state[1]*2}")
    log_info(f"  剩余时间 (Remaining Time): {state[2]/25}")
    log_info(f"  目标节点 (Target Node): {state[3]}")

def print_step_details(info):
    """
    打印每个步骤的详细信息。
    """
    log_info(f"  从节点 (From Node): {info['from_node']} 到节点 (To Node): {info['to_node']}")
    log_info(f"  距离缩短 (Distance_decrease): {info['distance_decrease']} km")
    log_info(f"  耗电 (Energy Consumed): {info['energy_consumed']} kWh")
    log_info(f"  是否充电 (Charging): {'Yes' if info['charging'] else 'No'}")
    if info['charging']:
        log_info(f"  充电量 (Energy Charged): {info['energy_charged']} kWh")
    log_info(f"  电量从 (Battery Start): {info['battery_start']} kWh 变成 (Battery End): {info['battery_end']} kWh")
    log_info(f"  剩余时间从 (Time Start): {info['time_start']} h 变成 (Time End): {info['time_end']} h")


# 测试训练好的模型
obs, info = env.reset()
done = False

log_info("Initial State:")
print_state(obs)
step = 0

# 执行100步动作
for i in range(100):
    # 预测动作
    action, _states = model.predict(obs, deterministic=True)
    # 执行动作
    obs, reward, done, done1, info = env.step(action)

    log_info("-" * 30)
    log_info(f"\nStep {step + 1}:")
    step += 1
    print_step_details(info)
    print_state(obs)
    log_info("Reward:", reward)
    log_info("Done:", done)

    if done:
        log_info("Episode finished! Resetting environment.")
        for i in range(50):
            obs, info = env.reset(seed=i)
        log_info("Initial State:" + "-" * 10)
        print_state(obs)
        step = 0

# 确保日志及时写入文件
logging.shutdown()

Initial State:
  当前节点 (Current Node): 43.0
  当前电池电量 (Battery): 48.0
  剩余时间 (Remaining Time): 1.6585298156738282
  目标节点 (Target Node): 41.0
------------------------------

Step 1:
  从节点 (From Node): 43 到节点 (To Node): 1
  距离缩短 (Distance_decrease): 6.218004805 km
  耗电 (Energy Consumed): 1.00859325912 kWh
  是否充电 (Charging): No
  电量从 (Battery Start): 48.0 kWh 变成 (Battery End): 46.99140674088 kWh
  剩余时间从 (Time Start): 1.658529921 h 变成 (Time End): 1.4675084704090908 h
  当前节点 (Current Node): 1.0
  当前电池电量 (Battery): 46.99140548706055
  剩余时间 (Remaining Time): 1.4675083923339844
  目标节点 (Target Node): 41.0
Reward: -102.6243268252
Done: False
------------------------------

Step 2:
  从节点 (From Node): 1 到节点 (To Node): 41
  距离缩短 (Distance_decrease): 5.523032765 km
  耗电 (Energy Consumed): 0.6627639318 kWh
  是否充电 (Charging): Yes
  充电量 (Energy Charged): 16.244214014705882 kWh
  电量从 (Battery Start): 46.99140674088 kWh 变成 (Battery End): 62.572856823785884 kWh
  剩余时间从 (Time Start): 1.4675084704090908 h 变成 

In [11]:
import logging
import os
import numpy as np
from env import ElectricVehicleEnv
from stable_baselines3 import A2C, DQN

# 设置绝对路径，确保文件生成在当前目录中
log_file_path = os.path.join(os.getcwd(), 'model_output.log')

# 配置日志，确保写入文件
logging.basicConfig(filename=log_file_path, level=logging.INFO, format='%(message)s', filemode='w')

def log_info(*args):
    """
    替代print函数，用logging.info来保存日志。
    """
    message = " ".join(map(str, args))
    print(message)
    logging.info(message)

def print_step_details(info, state, done):
    """
    打印步骤中需要的详细信息（从节点、当前电池电量、剩余时间、Done）。
    """
    from_node = info['from_node']
    battery = state[1] * 2  # 乘以2来获取电池电量
    remaining_time = state[2] / 25  # 除以25来计算剩余时间
    log_info(f"  从节点 (From Node): {from_node}")
    log_info(f"  当前电池电量 (Battery): {battery}")
    log_info(f"  剩余时间 (Remaining Time): {remaining_time}")
    log_info(f"  Done: {done}")


# 测试训练好的模型
obs, info = env.reset()
done = False

log_info("Initial State:")
step = 0

# 执行100步动作
for i in range(100):
    # 预测动作
    action, _states = model.predict(obs, deterministic=True)
    # 执行动作
    obs, reward, done, done1, info = env.step(action)

    log_info("-" * 30)
    log_info(f"\nStep {step + 1}:")
    step += 1
    print_step_details(info, obs, done)

    if done:
        log_info("Episode finished! Resetting environment.")
        for i in range(50):
            obs, info = env.reset(seed=i)
        log_info("Initial State:" + "-" * 10)
        step = 0

# 确保日志及时写入文件
logging.shutdown()

Initial State:
------------------------------

Step 1:
  从节点 (From Node): 30
  当前电池电量 (Battery): 54.47837829589844
  剩余时间 (Remaining Time): 1.4932463073730469
  Done: False
------------------------------

Step 2:
  从节点 (From Node): 20
  当前电池电量 (Battery): 51.727882385253906
  剩余时间 (Remaining Time): 0.9507420349121094
  Done: True
Episode finished! Resetting environment.
Initial State:----------
------------------------------

Step 1:
  从节点 (From Node): 24
  当前电池电量 (Battery): 31.59763526916504
  剩余时间 (Remaining Time): 1.1695079803466797
  Done: True
Episode finished! Resetting environment.
Initial State:----------
------------------------------

Step 1:
  从节点 (From Node): 26
  当前电池电量 (Battery): 31.990907669067383
  剩余时间 (Remaining Time): 1.4927639770507812
  Done: True
Episode finished! Resetting environment.
Initial State:----------
------------------------------

Step 1:
  从节点 (From Node): 31
  当前电池电量 (Battery): 71.5439224243164
  剩余时间 (Remaining Time): 1.3751304626464844
  Done: False

In [17]:
import pandas as pd

def extract_data_from_log(log_file):
    """
    从日志文件中提取每个车辆的路径、剩余电量和剩余截止时间信息。
    """
    car_data = []
    current_car = []
    
    with open(log_file, 'r', encoding='utf-8') as file:
        for line in file:
            if "从节点 (From Node)" in line:
                from_node = int(line.split(":")[1].strip())
            elif "当前电池电量 (Battery)" in line:
                battery = float(line.split(":")[1].strip())
            elif "剩余时间 (Remaining Time)" in line:
                remaining_time = float(line.split(":")[1].strip())
            elif "Done" in line:
                done = "True" in line
                
                # 添加当前步骤的数据
                current_car.append([from_node, battery, remaining_time])
                
                # 如果done为True，代表一辆车的数据结束，保存该车的数据
                if done:
                    car_data.append(current_car)
                    current_car = []
                    
    return car_data

def create_excel_from_data(car_data, output_file):
    """
    将提取的数据转换为 Excel 文件，每辆车的数据作为一个单独的表格。
    """
    with pd.ExcelWriter(output_file) as writer:
        for i, car_steps in enumerate(car_data, 1):
            # 创建DataFrame并添加到Excel中
            df = pd.DataFrame(car_steps, columns=['路径（点集）', '剩余电量（kwh）', '剩余截止时间（hour）'])
            df.index = range(1, len(df) + 1)
            df.to_excel(writer, sheet_name=f'车辆_{i}', index=False)

def main(log_file_path, output_excel_path):
    # 从日志文件中提取数据
    car_data = extract_data_from_log(log_file_path)
    
    # 创建Excel文件
    create_excel_from_data(car_data, output_excel_path)
    print(f"文件已保存到: {output_excel_path}")

# 使用示例
log_file = 'model_output.log'  # 替换为你的日志文件路径
output_excel = 'electric_vehicles.xlsx'  # 替换为你想要保存的Excel文件路径

main(log_file, output_excel)

文件已保存到: electric_vehicles.xlsx
